# Grokking Deeper: Where did the Date Come From?

## Setting Up
Here, we set up the variables we need from previous chapter's notebook in order to contiinue using them in this chapter. We just need the `clean_data_of_interest` variable to be available.

In [ ]:
%matplotlib inline
import pandas as pd

data = pd.read_csv("../datasets/PimaIndiansDiabetes.csv")
data_of_interest = data.loc[:, ["Blood Glucose", "BMI", "Class"]]

bmi_zeros_mask = data_of_interest.loc[:, "BMI"] != 0
bg_zeros_mask = data_of_interest.loc[:, "Blood Glucose"] != 0

clean_data_of_interest = data_of_interest[bmi_zeros_mask & bg_zeros_mask]

---
## 3.1 Probability and Distributions
Our first step in climbing out of the sample into the population is _normalize_ the count histograms. This is done by weighting the data with $\frac{1}{\text{sample_size}}$, hence treating transforming the counts into _ratios_ that we can interpret as **probabilities**. 

In [ ]:
sample_size = len(clean_data_of_interest.loc[:, "BMI"])
weights_list = [1 / sample_size] * sample_size

clean_data_of_interest.loc[:, "BMI"].plot.hist(weights=weights_list)

---
Combining this normalization process of counts with the density approach we saw in the previous chapter, we can get a histogram distribution that is _nearly_ free of any ties to the limited sample, a distribution of _**probability density**_. Instead of weighting the data of $\frac{1}{\text{sample_size}\times\text{bin_size}}$ to acheive that, we simply set the `normed` parameter in the `hist()` method to `True` for complete normalization.

In [ ]:
clean_data_of_interest.loc[:, "BMI"].plot.hist(normed=True) \
.set_ylabel("Probability Density")

---
## 3.2 Conditional Probability

<h4 style='color:darkred'>Exercise 3.3 </h4>

<span style='color:darkred'>Using our PIMA Indian diabetes data, try to estimate the conditional probability $P(BG > 140 | Class = 1)$.</span>

In [ ]:
# Your solution goes here

---
## 3.3 Applying the Naive Bayes Model with scikit-learn
Before we can apply the naive bayes formula, we need to decide how we're going to model $P(X_1|Y)$ and $P(X_2|Y)$. We can do that by isolating the features for the diabetic records and the non-diabetic records and see how these isolated groups are distributed 

In [ ]:
diabetics_mask = clean_data_of_interest.loc[:, "Class"] == 1

diabetics = clean_data_of_interest[diabetics_mask]
non_diabetics = clean_data_of_interest[~diabetics_mask]

Now we can use the `hist()` and `density()` method to see how these data are distributed and how does the approximate PDF looks like

In [ ]:
diabetics.loc[:, "BMI"].plot.hist(color='red', normed=True)
diabetics.loc[:, "BMI"].plot.density(color='black')

In [ ]:
non_diabetics.loc[:, "BMI"].plot.hist(color='green', normed=True)
non_diabetics.loc[:, "BMI"].plot.density(color='black')

---
<h4 style='color:darkred'>Exercise 3.4 </h4>

<span style='color:darkred'>Verify that the Blood Glucose conditional distribution $X_2|Y$ can be modeled with a normal distribution.</span>

In [ ]:
# Your solution goes here

---
Now that we know that the conditional distributions $X_1|Y$ and $X_2|Y$ can be modeld with a normal distributions, we're ready to apply our naive Bayes formula. We'll rely here on scikit-learn's `GaussianNB` to do all the parameter estimations for us and use the naive Bayes formula to predict whether a patient is diabetic or not given there BMI and BG measurments. We'll of course first split our data into training and testing set to be able to measure how accurate the naive Bayes model is.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

X = clean_data_of_interest.loc[:, ["Blood Glucose", "BMI"]]
y = clean_data_of_interest.loc[:, "Class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

classifier = GaussianNB()
classifier.fit(X_train, y_train)

accuracy = classifier.score(X_test, y_test)
print("Prediction Accuracy: {:.2f}%".format(accuracy * 100))

---
<h4 style='color:darkred'>Exercise 3.5 </h4>

<span style='color:darkred'>Notice that the data here changed from when we tried out the perceptron model in chapter one, due to the cleaning of missing values that happened in chapter two. Run the perceptron model again as well as the dummy classifier to make the results comparable with what we've got from the naive Bayes model.</span>

In [ ]:
# Your solution goes here

---
<h4 style='color:darkred'>Exercise 3.6 </h4>

<span style='color:darkred'>Try to implement the naive Bayes model on the traveling diabetes clinic problem yourself. It'll strengthen your understanding of how the different parts of the model work and how the estimation is done, as well as it'll develop a sense of appreciation of the work done behind scikit-learn and its supporting libraries. To help you in that implementation, note that the Bayes rule doesn't only apply to probabilities, but it also applies on PDF and PMF as well, so you can write the model for our problem as:
.</span>

$$
f_{Y|X_1,X_2}(Y|X_1, X_2) = \frac{f_{X_1|Y}(X_1|Y)f_{X_2|Y}(X_2|Y)f_Y(Y)}{f_{X_1}(X_1)f_{X_2}(X_2)}
$$

In [ ]:
# Your solution goes here